### Send players

In [ ]:
# send_players_to_thamm_and_log.py
import asyncio
import httpx
import json
import time

API_URL = "http://localhost:8000/join-queue"

async def send_player(session, player, ts_map):
    timestamp = time.time()
    ts_map[player["id"]] = timestamp  # save locally
    try:
        response = await session.post(API_URL, json=player)
        if response.status_code != 200:
            print(f"[ERROR] Player {player['id']} failed: {response.text}")
    except Exception as e:
        print(f"[EXCEPTION] Player {player['id']} failed: {e}")

async def main():
    with open("players.json") as f:
        players = json.load(f)

    ts_map = {}
    async with httpx.AsyncClient() as session:
        tasks = [send_player(session, player, ts_map) for player in players]
        for i in range(0, len(players), 100):
            await asyncio.gather(*tasks[i:i+100])
            print(f"Sent {i + 100} players...")

    # Save player send timestamps
    with open("player_timestamps.json", "w") as f:
        json.dump(ts_map, f, indent=2)

asyncio.run(main())

### Collect results

In [ ]:
# collect_results.py
from kafka import KafkaConsumer
import json
import time

consumer = KafkaConsumer(
    'match-results-1',
    bootstrap_servers='localhost:9092',
    auto_offset_reset='earliest',
    enable_auto_commit=True,
    group_id='thamm-result-analyzer',
    value_deserializer=lambda m: json.loads(m.decode('utf-8'))
)

results = []
start_time = time.time()
timeout = 30  # seconds

print("Listening to match-results...")

while time.time() - start_time < timeout:
    msg_pack = consumer.poll(timeout_ms=1000)  # wait up to 1 second for messages
    for tp, messages in msg_pack.items():
        for message in messages:
            match = message.value
            match['received_timestamp'] = time.time()
            results.append(match)
            print(f"Match: {match}")

with open("skill_match_results.json", "w") as f:
    json.dump(results, f, indent=2)

print("Done collecting results.")

Listening to match-results...
Match: {'matchId': '4d78be3f-e3df-4e9f-aab8-cb62ddd1eaf6', 'player1': {'id': 4, 'mmr': 2170, 'region': 'AF', 'churnLikelihood': 0.0, 'winCount': 27, 'lossCount': 20, 'tieCount': 2, 'lastGameResult': 2, 'streak': 7}, 'player2': {'id': 2, 'mmr': 2128, 'region': 'NA', 'churnLikelihood': 0.0, 'winCount': 12, 'lossCount': 16, 'tieCount': 1, 'lastGameResult': 0, 'streak': 6}, 'received_timestamp': 1749710031.4390092}
Match: {'matchId': '9231093b-11b3-4f49-82a6-4cd3c87c0bf6', 'player1': {'id': 6, 'mmr': 1463, 'region': 'ASIA', 'churnLikelihood': 0.0, 'winCount': 17, 'lossCount': 32, 'tieCount': 3, 'lastGameResult': 0, 'streak': 30}, 'player2': {'id': 7, 'mmr': 1429, 'region': 'ASIA', 'churnLikelihood': 0.0, 'winCount': 9, 'lossCount': 11, 'tieCount': 1, 'lastGameResult': 2, 'streak': 2}, 'received_timestamp': 1749710031.4403107}
Match: {'matchId': 'f3b6b8f1-1ee1-4c05-88b4-128a57fd48e7', 'player1': {'id': 3, 'mmr': 1384, 'region': 'AF', 'churnLikelihood': 0.0, 'wi

## Analysis

In [ ]:
# analyze_results.py
import json
from collections import defaultdict
import matplotlib.pyplot as plt

# Load data
with open("match_results.json") as f:
    matches = json.load(f)

with open("players.json") as f:
    players = {p["id"]: p for p in json.load(f)}

with open("player_timestamps.json") as f:
    player_send_times = json.load(f)

In [ ]:
# Metrics
latencies = []
elo_deltas = []
throughput_counts = defaultdict(int)
match_size_distribution = []

for match in matches:
    player_ids = match['players']
    timestamp_list = [players[pid]['timestamp'] for pid in player_ids if pid in players]

    if timestamp_list:
        latency = match['received_timestamp'] - min(timestamp_list)
        latencies.extend([latency] * len(player_ids))  # per player

    if "mmrs" in match:
        mmrs = match["mmrs"]
        delta = max(mmrs) - min(mmrs)
        elo_deltas.append(delta)

    ts = int(match['received_timestamp'])
    throughput_counts[ts] += 1

    match_size_distribution.append(len(player_ids))

# Throughput (matches per second)
throughput = len(matches) / 60  # assuming 60 seconds collection

# Plotting code (optional for notebook)
plt.hist(latencies, bins=50)
plt.title("Player Latency Distribution")
plt.xlabel("Latency (seconds)")
plt.ylabel("Player Count")
plt.show()

plt.hist(elo_deltas, bins=50)
plt.title("ELO Delta Distribution")
plt.xlabel("Delta")
plt.ylabel("Match Count")
plt.show()

plt.hist(match_size_distribution)
plt.title("Match Size Distribution")
plt.xlabel("Players per Match")
plt.ylabel("Frequency")
plt.show()

# Print summary
print(f"Average latency per player: {sum(latencies)/len(latencies):.2f} seconds")
print(f"Average ELO delta per match: {sum(elo_deltas)/len(elo_deltas):.2f}")
print(f"Throughput: {throughput:.2f} matches/sec")